# Hail PCA Exploration on HGDP + 1KG data

In this quick exploration of the HGDP + 1KG data, I explore some of the functions I will be using later on for different datasets. While this pipeline is not comprehensive in it's QC, it mainly serves as an exploration of the data and a way to become more familiar with Hail.

In [1]:
from bokeh.models import CategoricalColorMapper
from bokeh.palettes import turbo
from pprint import pprint
from gnomad.utils.annotations import annotate_adj
from hail.plot import show

Hail is a Python library capable of analysing (very) large datasets and is built on top of Apache Spark.

In [2]:
import hail as hl
hl.init()
hl.plot.output_notebook()

Running on Apache Spark version 2.4.5
SparkUI available at http://my-cluster-m.australia-southeast1-a.c.katalina-dev.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /home/hail/hail-20210304-2318-0.2.62-84fa81b9ea3d.log


Loading BokehJS ...

The dataset we'll be exploring is the gnomAD HGDP + 1KG callset, which is available as a Hail matrix table from [here](https://gnomad.broadinstitute.org/downloads#v3-hgdp-1kg). 

In [ ]:
mt = hl.read_matrix_table("gs://gcp-public-data--gnomad/release/3.1/mt/genomes/gnomad.genomes.v3.1.hgdp_1kg_subset_dense.mt")

In [ ]:
Let's first explore the Hail matrix table by seeing how many rows (variants) and columns (samples) we have. 

In [4]:
mt.count_rows()

175312130

In [5]:
mt.count_cols()

3942

We can also explore the Hail matrix table on the columns and rows by using the show() command

In [27]:
mt.cols().show()

+-----------+---------------------------+--------------------------+---------------------+---------------------------+
| s         | bam_metrics.pct_bases_20x | bam_metrics.pct_chimeras | bam_metrics.freemix | bam_metrics.mean_coverage |
+-----------+---------------------------+--------------------------+---------------------+---------------------------+
| str       |                   float64 |                  float64 |             float64 |                   float64 |
+-----------+---------------------------+--------------------------+---------------------+---------------------------+
| "HG00096" |                  9.55e+01 |                 6.91e-01 |            1.76e-06 |                  3.29e+01 |
| "HG00097" |                  9.47e+01 |                 5.99e-01 |            3.84e-09 |                  3.15e+01 |
| "HG00099" |                  9.62e+01 |                 5.97e-01 |            3.92e-09 |                  3.64e+01 |
| "HG00100" |                  9.41e+01 |                 6.08e-01 |            8.33e-09 |                  3.02e+01 |
| "HG00101" |                  9.53e+01 |                 6.92e-01 |            2.58e-06 |                  3.28e+01 |
| "HG00102" |                  9.45e+01 |                 6.00e-01 |            1.26e-08 |                  3.10e+01 |
| "HG00103" |                  9.47e+01 |                 6.47e-01 |            3.88e-06 |                  3.14e+01 |
| "HG00105" |                  9.19e+01 |                 5.52e-01 |            4.62e-06 |                  2.84e+01 |
| "HG00106" |                  9.60e+01 |                 5.95e-01 |            3.77e-09 |                  3.49e+01 |
| "HG00107" |                  9.50e+01 |                 5.77e-01 |            1.20e-08 |                  3.19e+01 |
+-----------+---------------------------+--------------------------+---------------------+---------------------------+

+-----------------------------+------------------------------+--------------------------------+---------------------------+-------------+
| bam_metrics.median_coverage | bam_metrics.mean_insert_size | bam_metrics.median_insert_size | bam_metrics.pct_bases_10x | subsets.tgp |
+-----------------------------+------------------------------+--------------------------------+---------------------------+-------------+
|                     float64 |                      float64 |                        float64 |                   float64 |        bool |
+-----------------------------+------------------------------+--------------------------------+---------------------------+-------------+
|                    3.30e+01 |                     4.33e+02 |                       4.23e+02 |                  9.72e+01 |        true |
|                    3.20e+01 |                     4.43e+02 |                       4.34e+02 |                  9.71e+01 |        true |
|                    3.70e+01 |                     4.41e+02 |                       4.31e+02 |                  9.72e+01 |        true |
|                    3.10e+01 |                     4.32e+02 |                       4.22e+02 |                  9.71e+01 |        true |
|                    3.30e+01 |                     4.54e+02 |                       4.45e+02 |                  9.72e+01 |        true |
|                    3.10e+01 |                     4.49e+02 |                       4.40e+02 |                  9.71e+01 |        true |
|                    3.20e+01 |                     4.24e+02 |                       4.15e+02 |                  9.71e+01 |        true |
|                    2.90e+01 |                     4.43e+02 |                       4.33e+02 |                  9.70e+01 |        true |
|                    3.50e+01 |                     4.39e+02 |                       4.30e+02 |                  9.72e+01 |        true |
|                    3.20e+01 |                     4.31e+02 |                       4.21e+02 |                  9.71e+01 |        true |
+------------------------

In [72]:
mt.rows().show()

+---------------+----------------+----------------+------------+------------------------+------------------------------------------+
| locus         | alleles        | rsid           | AS_lowqual | telomere_or_centromere | cohort_freq                              |
+---------------+----------------+----------------+------------+------------------------+------------------------------------------+
| locus<GRCh38> | array<str>     | str            |       bool |                   bool | array<struct{AC: int32, AF: float64, ... |
+---------------+----------------+----------------+------------+------------------------+------------------------------------------+
| chr1:10055    | ["T","C"]      | NA             |      false |                  false | [(0,0.00e+00,3190,0),(1,2.80e-04,3574... |
| chr1:10061    | ["T","C"]      | NA             |      false |                  false | [(0,0.00e+00,3480,0),(1,2.67e-04,3740... |
| chr1:10109    | ["A","T"]      | "rs376007522"  |      false |                  false | [(0,0.00e+00,346,0),(1,1.83e-03,546,0... |
| chr1:10109    | ["AACCCT","A"] | "rs1462685959" |      false |                  false | [(8,2.30e-02,348,0),(18,3.30e-02,546,... |
| chr1:10114    | ["T","C"]      | "rs1570391787" |      false |                  false | [(0,0.00e+00,450,0),(1,1.81e-03,552,0... |
| chr1:10114    | ["TA","T"]     | "rs1570391788" |      false |                  false | [(1,2.21e-03,452,0),(1,1.81e-03,552,0... |
| chr1:10116    | ["A","G"]      | NA             |      false |                  false | [(1,1.66e-04,6008,0),(1,1.41e-04,7104... |
| chr1:10119    | ["CT","C"]     | "rs1156821933" |      false |                  false | [(0,0.00e+00,6388,0),(1,1.37e-04,7320... |
| chr1:10120    | ["T","A"]      | NA             |      false |                  false | [(0,0.00e+00,292,0),(1,3.03e-03,330,0... |
| chr1:10122    | ["A","G"]      | NA             |      false |                  false | [(1,1.71e-04,5864,0),(1,1.51e-04,6634... |
+---------------+----------------+----------------+------------+------------------------+------------------------------------------+

+------------------------------------------+------------------------------------------+------------------------------------------+
| gnomad_freq                              | gnomad_raw_qual_hists.gq_hist_all.bin... | gnomad_raw_qual_hists.gq_hist_all.bin... |
+------------------------------------------+------------------------------------------+------------------------------------------+
| array<struct{AC: int32, AF: float64, ... | array<float64>                           | array<int64>                             |
+------------------------------------------+------------------------------------------+------------------------------------------+
| [(1,1.06e-05,94152,0),(5,4.65e-05,107... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [1429,1382,1901,2026,25677,6108,5433,... |
| [(0,0.00e+00,108768,0),(2,1.70e-05,11... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [728,922,1376,1510,23365,7233,6583,47... |
| [(0,0.00e+00,1556,0),(1,1.51e-04,6624... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [631,931,450,481,370,105,103,51,38,37... |
| [(107,6.91e-02,1548,0),(149,2.25e-02,... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [631,931,450,481,370,105,103,51,38,37... |
| [(5,2.25e-04,22208,0),(18,3.59e-04,50... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [2696,4495,3407,3311,6686,1542,1459,6... |
| [(4,1.80e-04,22206,0),(19,3.79e-04,50... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [2696,4495,3407,3311,6686,1542,1459,6... |
| [(0,0.00e+00,87160,0),(1,6.93e-06,144... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [3924,8266,6947,9422,20044,6216,7654,... |
| [(2,2.08e-05,96160,0),(18,1.22e-04,14... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [3837,7918,6439,7602,22553,6368,8045,... |
| [(0,0.00e+00,34092,0),(2,3.10e-05,644... | [0.00e+00,5.00e+00,1.00e+01,1.50e+01,... | [3124,5078,3825,3125,10735,2244,2081,... |
| [(0,0.00e+00,96128,0),(10,6.95e-05,

We can see that there is covariate information and several QC metrics already embedded within the matrix table. Additionally, we can perform basic QC metrics using the hl.sample_qc() command. Let's first look at the call rate (the fraction of all genotypes called) for each individual.

In [5]:
mt_qc = hl.sample_qc(mt)
p = hl.plot.histogram(mt_qc.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

We can see that there are some outlier individuals with low call rates. From what I've read so far, choosing a call rate threshold is pretty variable, but [95% or higher](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3025522/) seems to be pretty good. In the gnomAD QC, a hard filter of call rates < 0.895 was given.

In addition to call rates, we can also check out gene quality. Let's view the mean gene quality for each individual.

In [6]:
p_2 = hl.plot.histogram(mt_qc.sample_qc.gq_stats.mean, legend='Mean Sample GQ')
show(p_2)

We could (and should) filter out the low quality samples by, for example, removing samples that have a call rate less than 0.895 (as in the gnomadQC pipeline), however I found that after transforming the hail table matric to include the QC metrics, it's actually very computationally expensive to work on it (I still have yet to figure out why this is). So for now, we won't filter and will instead just practice using PCA on the data. If we were to filter, the command would be `mt = mt_qc.filter_cols((mt_qc.sample_qc.call_rate >= 0.895))`

## Making a PCA using Hail

Stored in the Hail matrix table are the results from running `hwe_normalized_pca`, a function which performs PCA analysis on the Hardy-Weinberg normalised data. I explored how long this would take for me to run on the dataset and, if using 100 VMs, it would take over 6 hours to complete (and nearly $310). Not worth it for exploratory purposes. The downside is that I don't have results that I would like to explore, such as the percentage of variance explained, but this is good enough for now.

We'd like to explore the PCA by population to see if there is any observable grouping. We have the option of labelling points with our PCA by sub-populations, however there are 73 subpopulations in all, which would be really hard to make sense of. Instead, we'll plot by the (broader) population parameter, which is stored in columns of the matrix table.

In [13]:
mt.cols().population_inference.pop.show()

2021-03-04 09:13:09 Hail: INFO: Coerced sorted dataset


,
s,
str,str
"""HG00096""","""nfe"""
"""HG00097""","""nfe"""
"""HG00099""","""nfe"""
"""HG00100""","""nfe"""
"""HG00101""","""nfe"""
"""HG00102""","""nfe"""
"""HG00103""","""nfe"""
"""HG00105""","""nfe"""


Let's assign the information we need to conduct a PCA to variables. Later, we'll be using labels and pca score information, and I found there's an error if the tables are not from the same object. To bypass this, we can assign mt.cols to a variable, then divide this between pca scores and label names. 

In [15]:
columns = mt.cols()
pca_scores = columns.population_inference.pca_scores
labels = columns.population_inference.pop

2021-03-05 00:53:14 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


For colours, we can use the categorical colour mapper from Bokeh. To do this, we need to get our populations as a string. 

In [17]:
pops = list(set(labels.collect()))
pops

2021-03-05 00:55:06 Hail: INFO: Coerced sorted dataset


['oth', 'amr', 'fin', 'mid', 'afr', 'sas', 'eas', 'nfe']

Now we can add the pop variable to factors within the CategoricalColorMapper function. For the palette, I went ahead and chose turo, simply because the colour scheme is qualitative.

In [18]:
mapper = CategoricalColorMapper(palette=turbo(8), factors=pops)

Let's first plot PC1 versus PC2

In [19]:
p = hl.plot.scatter(pca_scores[0], pca_scores[1], label=labels,
                    title='PCA', xlabel='PC1', ylabel='PC2', collect_all=True, colors = mapper)
show(p)

2021-03-05 00:57:40 Hail: INFO: Coerced sorted dataset


Neat! We can see a clear separation between distant populations, with African and other populations driving differences between PC1 and East Asians driving differences in PC2. Let's go through some of the PCs to explore any other trends.

In [20]:
p = hl.plot.scatter(pca_scores[1], pca_scores[2], label=labels,
                    title='PCA', xlabel='PC2', ylabel='PC3', collect_all=True, colors = mapper)
show(p)

2021-03-05 00:59:43 Hail: INFO: Coerced sorted dataset


We can see more (and different) clustering in PC2 vs PC3. Interestingly, American samples start to separate along PC3. 

In [21]:
p = hl.plot.scatter(pca_scores[2], pca_scores[3], label=labels,
                    title='PCA', xlabel='PC3', ylabel='PC4', collect_all=True, colors = mapper)
show(p)

2021-03-05 01:01:56 Hail: INFO: Coerced sorted dataset


In [22]:
p = hl.plot.scatter(pca_scores[3], pca_scores[4], label=labels,
                    title='PCA', xlabel='PC4', ylabel='PC5', collect_all=True, colors = mapper)
show(p)

2021-03-05 01:02:17 Hail: INFO: Coerced sorted dataset


In [23]:
p = hl.plot.scatter(pca_scores[4], pca_scores[5], label=labels,
                    title='PCA', xlabel='PC5', ylabel='PC6', collect_all=True, colors = mapper)
show(p)

2021-03-05 01:02:41 Hail: INFO: Coerced sorted dataset


Another thing I want to explore is whether any visible clustering is associated with other non-ancestry covariates. From the metadata, one obvious thing to check is whether clustering is driven by study (i.e., HGDP or 1KG). This data is boolean, and colour variables need to be strings. Therefore, this also presents a nice exercise to learn how to convert boolean characters to strings in Hail.

In [24]:
labels = columns.subsets.hgdp
hgdp = list(set(labels.collect()))
labels = hl.str(labels)

2021-03-05 01:08:59 Hail: INFO: Coerced sorted dataset


Now let's plot a few PCs and see. 

In [25]:
p = hl.plot.scatter(pca_scores[0], pca_scores[1], label=labels,
                    title='PCA', xlabel='PC1', ylabel='PC2', collect_all=True)
show(p)

2021-03-05 01:09:32 Hail: INFO: Coerced sorted dataset


In [26]:
p = hl.plot.scatter(pca_scores[1], pca_scores[2], label=labels,
                    title='PCA', xlabel='PC2', ylabel='PC3', collect_all=True)
show(p)

2021-03-05 01:10:40 Hail: INFO: Coerced sorted dataset


In [27]:
p = hl.plot.scatter(pca_scores[2], pca_scores[3], label=labels,
                    title='PCA', xlabel='PC3', ylabel='PC4', collect_all=True)
show(p)

2021-03-05 01:10:48 Hail: INFO: Coerced sorted dataset


In [28]:
p = hl.plot.scatter(pca_scores[3], pca_scores[4], label=labels,
                    title='PCA', xlabel='PC4', ylabel='PC5', collect_all=True)
show(p)

2021-03-05 01:10:53 Hail: INFO: Coerced sorted dataset


Great, from the plots, it doesn't appear that study is associated with the clustering we observe. One thing I would like to try out is actually getting the list of all covariates and performing an ANOVA association on the PCs and the covariates, although I imagine it might be computationally expensive. 